In [13]:
## Import relevant libraries
import numpy as np
from sklearn.naive_bayes import MultinomialNB
from nltk.tokenize import word_tokenize
from nltk import pos_tag
from nltk import download
import string
import re

In [ ]:
# Define some utilities

In [2]:
def process(tweet):
    # removes links, usernames, twitter special word 'RT' (retweet) and emoticons
    p = re.compile('(http://[^\s]+|@[^\s]*|RT|(\:\w+\:|\<[\/\\]?3|[\(\)\\\D|\*\$][\-\^]?[\:\;\=]|[\:\;\=B8][\-\^]?[3DOPp\@\$\*\\\)\(\/\|])(?=\s|[\!\.\?]|$))|&\w*')
    return p.sub("", tweet)

In [3]:
def sanitize(text):
    # removes punctuation character
    return text.translate(None, string.punctuation)

In [9]:
def removeStopWords(tokens):
    return [token for token in tokens if  token not in ["an", "a", "the"]]

In [10]:
def find_ngrams(input_list, n=2): 
  return zip(*[input_list[i:] for i in range(n)])

In [67]:
def nl():
    print("\n")

In [41]:
download("averaged_perceptron_tagger")
download("punkt")

[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /home/ds/nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!
[nltk_data] Downloading package punkt to /home/ds/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

In [42]:
tweet = "And now for something !!don't I'm completely different the http://google.fr"

# Feature extraction

### We remove URL links, Twitter user names and Twitter special words (e.g. "RT" (retweet)) and emoticons   

In [47]:
processed = process(tweet)
print ("Before: " + tweet)
print ("After: " + processed)

Before: And now for something !!don't I'm completely different the http://google.fr
After: And now for something !!don't I'm completely different the 


### Tokenization 

In [52]:
tokenized = word_tokenize(processed)
print ("Before: " + tweet)
print ("After: " + str(tokenized))

Before: And now for something !!don't I'm completely different the http://google.fr
After: ['And', 'now', 'for', 'something', '!', '!', 'do', "n't", 'I', "'m", 'completely', 'different', 'the']


### Removing stop words from bag of words

In [55]:
withoutStopWords = removeStopWords(tokenized)
print ("Before: " + tweet)
print ("After: " + str(withoutStopWords))

Before: And now for something !!don't I'm completely different the http://google.fr
After: ['And', 'now', 'for', 'something', '!', '!', 'do', "n't", 'I', "'m", 'completely', 'different']


### Compute Part-Of-Speech tags 

In [69]:
pos = pos_tag(withoutStopWords)
print ("Before: " + tweet)
print ("After: " + str(pos))

Before: And now for something !!don't I'm completely different the http://google.fr
After: [('And', 'CC'), ('now', 'RB'), ('for', 'IN'), ('something', 'NN'), ('!', '.'), ('!', '.'), ('do', 'VBP'), ("n't", 'RB'), ('I', 'PRP'), ("'m", 'VBP'), ('completely', 'RB'), ('different', 'JJ')]


### Remove punctuation

In [70]:
features_pos = [(k,v) for (k,v) in pos if k not in  string.punctuation ]
features_ngrams = find_ngrams([k for (k,_) in features_pos])
print ("Features set 1: " + str(features_pos))
nl()
print ("Features set 2: " + str(features_ngrams))

Features set 1: [('And', 'CC'), ('now', 'RB'), ('for', 'IN'), ('something', 'NN'), ('do', 'VBP'), ("n't", 'RB'), ('I', 'PRP'), ("'m", 'VBP'), ('completely', 'RB'), ('different', 'JJ')]


Features set 2: [('And', 'now'), ('now', 'for'), ('for', 'something'), ('something', 'do'), ('do', "n't"), ("n't", 'I'), ('I', "'m"), ("'m", 'completely'), ('completely', 'different')]


# Feature engineering (Please bring some coffee)